In [0]:
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional
from keras.datasets import imdb
import pandas as pd
import spacy
import numpy as np
import nltk
#nltk.download('punkt')
from nltk.tokenize import sent_tokenize, word_tokenize 
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
nlp = spacy.load('en')
from sklearn.model_selection import train_test_split
import pickle
from keras import backend as K
import matplotlib.pyplot as plt
from keras.optimizers import Adam
from keras.models import Model
from keras.layers import Input, Dense, Embedding, SpatialDropout1D, add, concatenate
from keras.layers import CuDNNLSTM, Bidirectional, GlobalMaxPooling1D, GlobalAveragePooling1D
from keras.preprocessing import text, sequence
from gensim.models import KeyedVectors

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
reviews_final_df = pd.read_csv('/content/drive/My Drive/Task2_Search/reviews_cleaned_filtered.csv')

In [6]:
reviews_final_df.head()

,cleaned_text_final,useful,class_useful
0,I disappointments recently Persis overall food...,2.0,0
1,BEHOLD THE GLORY OF A DELICIOUS PAN ROAST WITH...,2.0,0
2,Five bucks parking Nevada ID Once parking spot...,30.0,0
3,I little leery Chinese I let times worried I e...,2.0,0
4,My boyfriend I loved order delivery place late...,1.0,0


In [0]:
#train test split:
from sklearn.model_selection import train_test_split
train_df,test_df = train_test_split(reviews_final_df,test_size=0.3, random_state=42)

In [0]:
#train val split:
from sklearn.model_selection import train_test_split
training_df,val_df = train_test_split(train_df,test_size=0.2, random_state=42)

In [0]:
def build_matrix(word_index, path):
    embedding_index = KeyedVectors.load(path, mmap='r')
    embedding_matrix = np.zeros((len(word_index) + 1, 300))
    for word, i in word_index.items():
        for candidate in [word, word.lower()]:
            if candidate in embedding_index:
                embedding_matrix[i] = embedding_index[candidate]
                break
    return embedding_matrix

In [0]:
DENSE_HIDDEN_UNITS1 = 512
DENSE_HIDDEN_UNITS2 = 512
LSTM_UNITS1 = 256
LSTM_UNITS2 = 256
def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true)))

def r2_keras(y_true, y_pred):
    SS_res =  K.sum(K.square(y_true - y_pred)) 
    SS_tot = K.sum(K.square(y_true - K.mean(y_true))) 
    return (1 - SS_res/(SS_tot + K.epsilon()))


def build_model(embedding_matrix):
    words = Input(shape=(None,))

    x = Embedding(*embedding_matrix.shape, weights=[embedding_matrix], trainable=False)(words)
    drop1 = SpatialDropout1D(0.4)(x)
    lstm1 = Bidirectional(CuDNNLSTM(LSTM_UNITS1, return_sequences=True))(drop1)
    drop2 = Dropout(0.8)(lstm1)
    lstm2 = Bidirectional(CuDNNLSTM(LSTM_UNITS2, return_sequences=True))(drop2)

    hidden = concatenate([
        GlobalMaxPooling1D()(lstm2),
        GlobalAveragePooling1D()(lstm2),
    ])
    
    dense1 = Dense(DENSE_HIDDEN_UNITS1, activation='relu')(hidden)
    drop3 = Dropout(0.8)(dense1)

    result = Dense(1, activation='relu')(drop3)

    
    model = Model(inputs=words, outputs=result)
    model.compile(loss=root_mean_squared_error, optimizer=Adam(lr = 0.001),metrics = [r2_keras])

    return model

In [26]:
CHARS_TO_REMOVE = '!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n“”’\'∞θ÷α•à−β∅³π‘₹´°£€\×™√²—'

MAX_LEN = 220
EMBEDDING_FILES = [
    '/content/drive/My Drive/Task2_Search/crawl-300d-2M.gensim',
    '/content/drive/My Drive/Task2_Search/glove.840B.300d.gensim'
]
x_train = training_df["cleaned_text_final"].astype(str)
y_train = training_df["useful"].values
x_test = test_df["cleaned_text_final"].astype(str)
x_val = val_df["cleaned_text_final"].astype(str)
y_val = val_df["useful"].values

tokenizer = text.Tokenizer(filters=CHARS_TO_REMOVE, lower=False)
tokenizer.fit_on_texts(list(x_train) + list(x_test) +  list(x_val))

x_train = tokenizer.texts_to_sequences(x_train)
x_test = tokenizer.texts_to_sequences(x_test)
x_val = tokenizer.texts_to_sequences(x_val)

x_train = sequence.pad_sequences(x_train, maxlen=MAX_LEN)
x_test = sequence.pad_sequences(x_test, maxlen=MAX_LEN)
x_val = sequence.pad_sequences(x_val, maxlen=MAX_LEN)

embedding_matrix = np.concatenate(
    [build_matrix(tokenizer.word_index, f) for f in EMBEDDING_FILES], axis=-1)


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [33]:
model = build_model(embedding_matrix)
model.fit(x_train,y_train,batch_size=512,epochs=40,validation_data = (x_val, y_val))

#model.predict(x_test)

Train on 21863 samples, validate on 5466 samples
Epoch 1/40
21863/21863 [==============================] - 75s 3ms/step - loss: 4.1302 - r2_keras: 0.0036 - val_loss: 5.0046 - val_r2_keras: 0.0165
Epoch 2/40
21863/21863 [==============================] - 71s 3ms/step - loss: 4.0478 - r2_keras: 0.0441 - val_loss: 5.0631 - val_r2_keras: -0.0136
Epoch 3/40
21863/21863 [==============================] - 71s 3ms/step - loss: 4.0193 - r2_keras: 0.0478 - val_loss: 5.0796 - val_r2_keras: -0.0224
Epoch 4/40
21863/21863 [==============================] - 71s 3ms/step - loss: 4.0100 - r2_keras: 0.0617 - val_loss: 4.8924 - val_r2_keras: 0.0678
Epoch 5/40
21863/21863 [==============================] - 71s 3ms/step - loss: 3.9833 - r2_keras: 0.0730 - val_loss: 5.0053 - val_r2_keras: 0.0141
Epoch 6/40
21863/21863 [==============================] - 71s 3ms/step - loss: 3.9635 - r2_keras: 0.0809 - val_loss: 4.9662 - val_r2_keras: 0.0286
Epoch 7/40
21863/21863 [==============================] - 71s 3ms/s

KeyboardInterrupt: ignored

In [1]:
pred_test = model.predict(x_test)
y_test = test_df["useful"].values


NameError: ignored

In [2]:
test_rmse = np.sqrt(np.mean(np.square(pred_test - y_test)))
print("test_rmse",test_rmse)

NameError: ignored